# Precondition df

In [2]:
import os
import pandas as pd

In [3]:
ID_arr = [1,2,3,4,5,6]
day_arr = [1,2]
set = [1,2,3,4,5,6]

## HR Creation

In [ ]:
from datetime import datetime
for set_ in set:
    with pd.ExcelWriter(f'HR_set{set_}.xlsx', engine="xlsxwriter", datetime_format='yyyy-mm-dd hh:mm:ss.000') as writer:        
        for id_ in ID_arr:
            result = pd.DataFrame(columns=["Time","HR","RR","MS","SKINCONTACT","experiment_set", "experiment_id"])
            for day_ in day_arr:
                print(f"process set: {set_} day: {day_} id: {id_}")
                try:
                    csv_arr = os.listdir(f'HR/set{set_}/Day{day_}/ID{id_}/')
                except FileNotFoundError:
                    print(f"Not Found set: {set_} day: {day_} id: {id_}")
                    continue
                csv_arr = csv_arr[:len(csv_arr)//2]
                for i in range(len(csv_arr)):
                    csv_arr[i] = csv_arr[i][-20:-4]

                for csv_ in csv_arr:
                    result_temp = pd.DataFrame(columns=["Time","HR","RR","MS","SKINCONTACT","experiment_set", "experiment_id"])
                    file_name_HR = f'HR/set{set_}/Day{day_}/ID{id_}/HR_'+csv_+'.txt'
                    skiprows = 6
                    data_HR = pd.read_csv(file_name_HR, skiprows=skiprows, sep=" ", header=None, names=["CHANNEL","RR","MS","SKINCONTACT"], index_col=False)
                    
                    # break
                    timestamp = datetime.strptime(csv_, "%d.%m.%Y_%H.%M")
                    result_temp['Time'] = [timestamp]*len(data_HR)
                    result_temp['experiment_set'] = [set_]*len(data_HR)
                    result_temp['experiment_id'] = [id_]*len(data_HR)
                    result_temp['MS'] = pd.to_timedelta(data_HR['MS'],'ms')
                    result_temp['Time'] = result_temp['Time'] + result_temp['MS']
                    result_temp['HR'] = data_HR['CHANNEL']
                    result_temp['RR'] = data_HR['RR']
                    result_temp['SKINCONTACT'] = data_HR['SKINCONTACT']
                    frames = [result, result_temp]
                    result = pd.concat(frames)
                # break
            result.to_excel(writer, sheet_name=f"id{id_}", index=False)
            # display(result)
            # break
    # break

## ECG Creation

In [ ]:
from datetime import datetime
for set_ in set:
    print(f"process {set_}")
    # with pd.ExcelWriter(f'ECG_set{set_}.xlsx', engine="xlsxwriter", datetime_format='yyyy-mm-dd hh:mm:ss.000') as writer:        
    for id_ in ID_arr:
        result = pd.DataFrame(columns=["Time","ECG","MS","experiment_set", "experiment_id"])
        for day_ in day_arr:
            # print(f"process set: {set_} day: {day_} id: {id_}")
            try:
                csv_arr = os.listdir(f'HR/set{set_}/Day{day_}/ID{id_}/')
            except FileNotFoundError:
                print(f"Not Found set: {set_} day: {day_} id: {id_}")
                continue
            csv_arr = csv_arr[:len(csv_arr)//2]
            for i in range(len(csv_arr)):
                csv_arr[i] = csv_arr[i][-20:-4]

            for csv_ in csv_arr:
                result_temp = pd.DataFrame(columns=["Time","ECG","MS","experiment_set", "experiment_id"])
                file_name_HR = f'HR/set{set_}/Day{day_}/ID{id_}/ECG_'+csv_+'.txt'
                skiprows = 6
                data_ECG = pd.read_csv(file_name_HR, skiprows=skiprows, sep=" ", header=None, names=["CHANNEL","MS"], index_col=False)
                
                # break
                timestamp = datetime.strptime(csv_, "%d.%m.%Y_%H.%M")
                result_temp['Time'] = [timestamp]*len(data_ECG)
                result_temp['experiment_set'] = [set_]*len(data_ECG)
                result_temp['experiment_id'] = [id_]*len(data_ECG)
                result_temp['MS'] = pd.to_timedelta(data_ECG['MS'],'ms')
                result_temp['Time'] = result_temp['Time'] + result_temp['MS']
                result_temp['ECG'] = data_ECG['CHANNEL']
                frames = [result, result_temp]
                result = pd.concat(frames)
            # break
        result.to_csv(f'ECG_set{set_}_id{id_}.csv', index=False)
        # display(result)
        # break
    # break

# Insert the mean data

In [44]:
master_data = pd.read_excel("Experiment_edited_V4.xlsx")
slice_data = master_data[['End Date',"Experiment set",'Experiment id', "Time_thermal"]]
slice_data.head()

,End Date,Experiment set,Experiment id,Time_thermal
0,2023-07-10 11:02:50,set1,id2,10:45-11:05
1,2023-07-10 11:03:41,set1,id6,10:45-11:05
2,2023-07-10 11:03:52,set1,id1,10:45-11:05
3,2023-07-10 11:04:08,set1,id5,10:45-11:05
4,2023-07-10 11:05:05,set1,id3,10:45-11:05


In [138]:
data_set = {}
new_set = [1,2,3,4,5,6]
for set_ in new_set:
    data_set["set"+str(set_)]={}
    for id_ in ID_arr:
        data_set["set"+str(set_)]["id"+str(id_)] = pd.read_excel(f"SkinT_set{set_}.xlsx", sheet_name=f"id{id_}", parse_dates=['Time'], date_format="mixed")

In [139]:
from datetime import datetime
refineset = ["set1","set2"]
for set_ in refineset:
    for id_ in ID_arr:
        date_result = []
        # print(f"process on {set_} id{id_}")
        for _, row in data_set[set_]["id"+str(id_)].iterrows():
            date_ = row['Time']
            if date_.day == 13:
                date_result.append(date_)
                continue    
            date_ = datetime.strptime(str(date_), "%Y-%d-%m %H:%M:%S")
            date_result.append(date_)
        data_set[set_]["id"+str(id_)].Time=date_result

process on set1 id1
process on set1 id2
process on set1 id3
process on set1 id4
process on set1 id5
process on set1 id6
process on set2 id1
process on set2 id2
process on set2 id3
process on set2 id4
process on set2 id5
process on set2 id6


In [140]:
from datetime import datetime
# end_time = datetime.strptime("2023-07-10 11:05", "%Y-%m-%d %H:%M")
start_time = datetime.strptime("2023-07-10 10:45", "%Y-%m-%d %H:%M")
data_set["set1"]["id1"][data_set["set1"]["id1"]['Time']>=start_time]

,Time,Unit,1,2,3,4,Tmsk
46,2023-07-10 10:45:01,C,35.095,34.098,34.595,33.140,34.3049
47,2023-07-10 10:46:01,C,35.095,33.599,34.595,33.140,34.1552
48,2023-07-10 10:47:01,C,35.095,33.100,34.595,33.140,34.0055
49,2023-07-10 10:48:01,C,35.095,33.100,35.095,33.140,34.1055
50,2023-07-10 10:49:01,C,34.596,33.599,35.095,33.140,34.1055
...,...,...,...,...,...,...,...
979,2023-07-11 18:10:01,C,25.105,25.105,25.104,25.148,25.1134
980,2023-07-11 18:11:01,C,25.105,25.105,25.104,25.148,25.1134
981,2023-07-11 18:12:01,C,25.105,25.105,25.104,25.148,25.1134
982,2023-07-11 18:13:01,C,25.105,25.105,25.104,25.148,25.1134


In [141]:
from datetime import datetime
result_tmsk_avg = []
try:
    for _, row in slice_data.iterrows():
        date = str(row["End Date"].date())
        start_clock, end_clock = row['Time_thermal'].split("-")
        start_time = date+" "+start_clock
        end_time = date+" "+end_clock

        end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")

        # print(row)
        # print(start_time)
        # print(end_time)

        dataFiltered = data_set[row['Experiment set']][row['Experiment id']][(end_time > data_set[row['Experiment set']][row['Experiment id']]['Time']) & (start_time <= data_set[row['Experiment set']][row['Experiment id']]['Time'])]
        # print(dataFiltered)
        # break
        result_tmsk_avg.append(dataFiltered.Tmsk.mean(axis=0))
except ValueError:
    print(row) 

master_data['Average_tmsk'] = result_tmsk_avg

In [142]:
master_data.to_excel("test.xlsx", index=False)